# LSTM CAMELS-ES: comparativa
***

***Autor:** Jesús Casado Rodríguez*<br>
***Fecha:** 16-09-2023*<br>

**Introducción:**<br>
En este _notebook_ se comparan los distintos modelos LSTM entrenados sobre los datos de CAMELS-ES.

**Por hacer**:

In [1]:
import numpy as np
import pandas as pd
import os
import glob
from tqdm.notebook import tqdm
from pathlib import Path
from neuralhydrology.utils.config import Config

os.chdir('../')
from model_utils import *
os.chdir('CAMELS-ES/')

In [8]:
runs = [Path(x) for x in glob.glob('runs/CAMELS-ES*') if os.path.isdir(x)]
names = [Config(Path(run) / 'config.yml').experiment_name for run in runs]

compare = pd.DataFrame(index=names, columns=['train', 'validation', 'test'])
for run in tqdm(runs):
    cfg = Config(Path(run) / 'config.yml')
    name = cfg.experiment_name
    # extraer el rendimiento de cada época y muestra
    df = pd.DataFrame(columns=range(1, cfg.epochs + 1))
    for epoch in df.columns:
        try:
            _, df[epoch] = get_results(Path(run), 'validation', epoch=epoch)
        except:
            continue
    
    epoch = df.median().idxmax()
    print('{0:<20}\tmejor época: {1}\t\tKGE = {2:.3f}'.format(name, epoch, df.median().max()))
    
    for period in ['train', 'validation', 'test']:
        _, rend = get_results(Path(run), period, epoch=epoch)
        compare.loc[name, period] = rend.median().values[0]

  0%|          | 0/12 [00:00<?, ?it/s]

CAMELS-ES_base      	mejor época: 21		KGE = 0.577
CAMELS-ES_batch128  	mejor época: 30		KGE = 0.502
CAMELS-ES_batch526  	mejor época: 7		KGE = 0.561
CAMELS-ES_clip0.8   	mejor época: 22		KGE = 0.601
CAMELS-ES_clipNone  	mejor época: 30		KGE = 0.572
CAMELS-ES_drop03    	mejor época: 28		KGE = 0.549
CAMELS-ES_drop05    	mejor época: 30		KGE = 0.542
CAMELS-ES_end2      	mejor época: 14		KGE = 0.518
CAMELS-ES_end3      	mejor época: 12		KGE = 0.581
CAMELS-ES_end       	mejor época: 25		KGE = 0.591
CAMELS-ES_hidden032 	mejor época: 24		KGE = 0.544
CAMELS-ES_hidden128 	mejor época: 13		KGE = 0.593


In [9]:
compare

,train,validation,test
CAMELS-ES_base,0.819467,0.577075,0.590615
CAMELS-ES_batch128,0.762484,0.502029,0.544978
CAMELS-ES_batch526,0.713533,0.560649,0.497923
CAMELS-ES_clip0.8,0.838561,0.600601,0.560731
CAMELS-ES_clipNone,0.869142,0.57244,0.635542
CAMELS-ES_drop03,0.78183,0.549126,0.56006
CAMELS-ES_drop05,0.817583,0.54248,0.580649
CAMELS-ES_end2,0.816321,0.517593,0.613138
CAMELS-ES_end3,0.799748,0.580705,0.586751
CAMELS-ES_end,0.783734,0.591143,0.544006


Escojo como mejor modelo el llamado `CAMELS-ES_hidden128`.